## 해당에서는 인덱스로 만들고 컬럼자체가 한개라서 명을 안써줬는데 실제에서는 기입을 해준다.

### pandas 첫날 배운 형변환
- datetime 변환

In [1]:
import pandas as pd

# 센서 데이터
df = pd.DataFrame({
    'timestamp': ['2024-01-01 09:00', '2024-01-01 09:05', 
                  '2024-01-01 09:10', '2024-01-01 09:15'],
    'temperature': [35.0, 36.5, 38.0, 37.2]
})

print(df)
print(f'\n형변환 이전 데이터 타입: {df['timestamp'].dtype}')

#근데 모두가 날짜로 기록이 되어있기에 object보다는 날짜 타잎으로 바꾸는게 낫다 판단.
df['timestamp' ]= pd.to_datetime(df['timestamp'])
print(f'형변환 이후 데이터 타입: {df['timestamp'].dtype}')

          timestamp  temperature
0  2024-01-01 09:00         35.0
1  2024-01-01 09:05         36.5
2  2024-01-01 09:10         38.0
3  2024-01-01 09:15         37.2

형변환 이전 데이터 타입: object
형변환 이후 데이터 타입: datetime64[ns]


### 인덱스가 시간이 되면 시계열 기능을 사용할수 있다.
- datetime 인덱스 설정

In [2]:
df = df.set_index('timestamp')
print(df)

                     temperature
timestamp                       
2024-01-01 09:00:00         35.0
2024-01-01 09:05:00         36.5
2024-01-01 09:10:00         38.0
2024-01-01 09:15:00         37.2


### 시계열 함수 사용
- resample 
- rolling

In [3]:
# 🐚 resample은 시간용 groupby라 봐도 된다. 기준은 항상 00부터임 설령 데이터가 늦어도 00부터 계산이 들어가는것이다. 
df.resample('10min').mean()

,temperature
timestamp,
2024-01-01 09:00:00,35.75
2024-01-01 09:10:00,37.60


In [4]:
# 🐚 행수는 그대로. 대신 충족안하면 NaN. 최소 기준은 min_periods를 사용
df.rolling(3,min_periods=2).mean()

,temperature
timestamp,
2024-01-01 09:00:00,NaN
2024-01-01 09:05:00,35.750000
2024-01-01 09:10:00,36.500000
2024-01-01 09:15:00,37.233333


### 2

In [5]:
df = pd.DataFrame({
    'day': pd.date_range('2024-01-01', periods=7),
    'sales': [100, 120, 80, 150, 200, 180, 160]
})

print(df)
print(df['day'].dtype)

print('\n 인덱스 설정')
df = df.set_index('day')
print(df)

         day  sales
0 2024-01-01    100
1 2024-01-02    120
2 2024-01-03     80
3 2024-01-04    150
4 2024-01-05    200
5 2024-01-06    180
6 2024-01-07    160
datetime64[ns]

 인덱스 설정
            sales
day              
2024-01-01    100
2024-01-02    120
2024-01-03     80
2024-01-04    150
2024-01-05    200
2024-01-06    180
2024-01-07    160


In [6]:
df['3일_rolling'] = df.rolling(3).mean()

df

,sales,3일_rolling
day,,
2024-01-01,100,NaN
2024-01-02,120,NaN
2024-01-03,80,100.000000
2024-01-04,150,116.666667
2024-01-05,200,143.333333
2024-01-06,180,176.666667
2024-01-07,160,180.000000


### 연습 코드 
- 1시간 평균 , 2개의 이동 평균을 나타내기

In [7]:
import pandas as pd

# 테스트 데이터
df = pd.DataFrame({
    'timestamp': ['2024-01-01 09:00', '2024-01-01 09:30', 
                  '2024-01-01 10:00', '2024-01-01 10:30',
                  '2024-01-01 11:00', '2024-01-01 11:30'],
    'temperature': [35.0, 36.0, 38.0, 39.0, 37.0, 36.5]
})

In [8]:
# 🐚 시작할때 df의 전체적인 구성이랑 어떤 타입인지 먼저 확인하는 습관 들이기. 
df.info() # timestamp가 오브젝트 타입임 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestamp    6 non-null      object 
 1   temperature  6 non-null      float64
dtypes: float64(1), object(1)
memory usage: 228.0+ bytes


In [9]:
df.describe()

,temperature
count,6.000000
mean,36.916667
std,1.428869
min,35.000000
25%,36.125000
50%,36.750000
75%,37.750000
max,39.000000


In [10]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [11]:
df

,timestamp,temperature
0,2024-01-01 09:00:00,35.0
1,2024-01-01 09:30:00,36.0
2,2024-01-01 10:00:00,38.0
3,2024-01-01 10:30:00,39.0
4,2024-01-01 11:00:00,37.0
5,2024-01-01 11:30:00,36.5


In [12]:
# 🐚 이후에 시계열을 쓰고 싶다면 시간 관련 피쳐를 index로 해준다.
df = df.set_index('timestamp')
df

,temperature
timestamp,
2024-01-01 09:00:00,35.0
2024-01-01 09:30:00,36.0
2024-01-01 10:00:00,38.0
2024-01-01 10:30:00,39.0
2024-01-01 11:00:00,37.0
2024-01-01 11:30:00,36.5


In [13]:
# 1시간 평균 
# 🐚 기존 테이블에 붙이는거는 사실상 두개가 하나가 되서 기본값인 left 라벨에만 값이 존재하는것이다.
# 🐚 나머지는 NaN으로 표시 
df['1시간평균'] = df.resample('1h').mean()
print(df['1시간평균'])
print(df)

# 🐚 기존 테이블에 붙일때는 NaN을 방지하기 위해서는 transform으로 채우기도 가능하다.
df['1시간평균_전부채우기'] = df.resample('1h')['temperature'].transform('mean')
print(df['1시간평균_전부채우기']) 

timestamp
2024-01-01 09:00:00    35.50
2024-01-01 09:30:00      NaN
2024-01-01 10:00:00    38.50
2024-01-01 10:30:00      NaN
2024-01-01 11:00:00    36.75
2024-01-01 11:30:00      NaN
Name: 1시간평균, dtype: float64
                     temperature  1시간평균
timestamp                              
2024-01-01 09:00:00         35.0  35.50
2024-01-01 09:30:00         36.0    NaN
2024-01-01 10:00:00         38.0  38.50
2024-01-01 10:30:00         39.0    NaN
2024-01-01 11:00:00         37.0  36.75
2024-01-01 11:30:00         36.5    NaN
timestamp
2024-01-01 09:00:00    35.50
2024-01-01 09:30:00    35.50
2024-01-01 10:00:00    38.50
2024-01-01 10:30:00    38.50
2024-01-01 11:00:00    36.75
2024-01-01 11:30:00    36.75
Name: 1시간평균_전부채우기, dtype: float64


In [14]:
# 2개의 이동평균
# 🐚 df['temperture']로 rolling이 가능한 이유는 그냥 행을 하기 떄문 '1h' 같으면 시간타입으로만 가능 
df['2개_이동평균'] = df['temperature'].rolling(2).mean()
print(df)

                     temperature  1시간평균  1시간평균_전부채우기  2개_이동평균
timestamp                                                    
2024-01-01 09:00:00         35.0  35.50        35.50      NaN
2024-01-01 09:30:00         36.0    NaN        35.50    35.50
2024-01-01 10:00:00         38.0  38.50        38.50    37.00
2024-01-01 10:30:00         39.0    NaN        38.50    38.50
2024-01-01 11:00:00         37.0  36.75        36.75    38.00
2024-01-01 11:30:00         36.5    NaN        36.75    36.75
